## 128. 最长连续序列

In [4]:
from typing import List, Optional, Tuple, Set
from collections import defaultdict, Counter
from functools import lru_cache

In [5]:
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        nums, ans = set(nums), 0
        for n in nums:
            if n - 1 in nums: continue
            cur = n
            while cur + 1 in nums: cur += 1
            ans = max(ans, cur - n + 1)
        return ans

### 核心： 若 n - 1 也在集合中则跳过, 否则依次寻找连续的值。这样才能达到O(n)的复杂度

没有必要用并查集

## 2316. 统计无向图中无法相互到达点对数

In [57]:
class Solution:
    def countPairs(self, n: int, edges: List[List[int]]) -> int:
        def find(x):
            if fa[x] != x: fa[x] = find(fa[x])
            return fa[x]
        
        def union(x, y):
            fx, fy = find(x), find(y)
            if fx != fy:
                if rank[fx] > rank[fy]:
                    fx, fy = fy, fx
                fa[fx] = fy
                if rank[fx] == rank[fy]: rank[fy] += 1    
        
        fa = [i for i in range(n)]
        rank = [1] * n
        for a, b in edges: union(a, b)
        for i in range(n): fa[i] = find(i)
        nums, pres, cnt = [], [], Counter(fa)
        pre = 0
        for key in cnt: 
            pre += cnt[key]
            nums.append(cnt[key])
            pres.append(pre)
        ans = 0
        for i in range(len(nums)):
            ans += nums[i] * (pres[-1] - pres[i])
        
        
        
        # return nums, pres, ans
        return ans
        # return fa

In [58]:
n = 11
edges = [[5,0],[1,0],[10,7],[9,8],[7,2],[1,3],[0,2],[8,5],[4,6],[4,2]]

In [59]:
s = Solution()
s.countPairs(n, edges)

0

## 130. 被围绕的区域

换个角度, 就是看联通的"O"区域是否能够到达边界, 不能到达的都变成 "X"

In [61]:
class UF:
    def __init__(self, n, r, c):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
        self.lb = [-1] * n
        self.hb = [-1] * n
        self.r = r
        self.c = c
        
        
    def find(self, x): 
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            self.fa[fx] = fy
            if self.rank[fx] == self.rank[fy]: self.rank[fy] += 1
            row, col = divmod(fx, self.c)
            self.lb[fy] = min(self.lb[fy], min(row, col))
            self.hb[fy] = max(self.hb[fy], max(row, col))
            
            
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        r, c = len(board), len(board[0])
        uf = UF(r * c, r, c)
        for i in range(r):
            for j in range(c):
                if board[i][j] == "O":
                    x = i * r + j
                    for di, dj in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                        ni, nj = i + di, j + dj
                        if 0 <= ni < r and 0 <= nj < c:
                            if board[ni][nj] == "O":
                                y = ni * r + nj
                                uf.union(x, y)
        
        return uf.fa, uf.lb, uf.hb

In [62]:
board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]

In [63]:
s = Solution()
s.solve(board)

([0, 1, 2, 3, 4, 6, 6, 7, 8, 9, 6, 11, 12, 13, 14, 15],
 [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
 [-1, -1, -1, -1, -1, -1, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1])

这题用并查集有些麻烦，而且用了本质上还是要dfs或bfs。

**思路：遍历一遍矩阵，以在边界上的 “O” 为起点的 “O” 区域标记。再次遍历，若遇到没有标记的 “O” 则把他变成 “X”**

### DFS

In [69]:
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        """
        Do not return anything, modify board in-place instead.
        """
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if board[nx][ny] == "O" and mark[nx][ny] == 0:
                        mark[nx][ny] = 1
                        dfs(nx, ny)
        
        m, n = len(board), len(board[0])
        mark = [[0] * n for _ in range(m)]
        for i in range(m):
            if board[i][0] == "O":
                mark[i][0] = 1
                dfs(i, 0)
            if board[i][n - 1] == "O":
                mark[i][n - 1] = 1
                dfs(i, n - 1)
        for i in range(n):
            if board[0][i] == "O":
                mark[0][i] = 1
                dfs(0, i)
            if board[m - 1][i] == "O":
                mark[m - 1][i] = 1
                dfs(m - 1, i)
        
        for i in range(m):
            for j in range(n):
                if board[i][j] == "O" and mark[i][j] == 0:
                    board[i][j] = "X"
        
        return board

In [70]:
board = [["X","X","X","X"],["X","O","O","X"],["X","X","O","X"],["X","O","X","X"]]

In [71]:
s = Solution()
s.solve(board)

[['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'X', 'X', 'X'],
 ['X', 'O', 'X', 'X']]

### BFS

In [72]:
class Solution:
    def solve(self, board: List[List[str]]) -> None:
        if not board:
            return
        
        n, m = len(board), len(board[0])
        que = collections.deque()
        for i in range(n):
            if board[i][0] == "O":
                que.append((i, 0))
                board[i][0] = "A"
            if board[i][m - 1] == "O":
                que.append((i, m - 1))
                board[i][m - 1] = "A"
        for i in range(m - 1):
            if board[0][i] == "O":
                que.append((0, i))
                board[0][i] = "A"
            if board[n - 1][i] == "O":
                que.append((n - 1, i))
                board[n - 1][i] = "A"
        
        while que:
            x, y = que.popleft()
            for mx, my in [(x - 1, y), (x + 1, y), (x, y - 1), (x, y + 1)]:
                if 0 <= mx < n and 0 <= my < m and board[mx][my] == "O":
                    que.append((mx, my))
                    board[mx][my] = "A"
        
        for i in range(n):
            for j in range(m):
                if board[i][j] == "A":
                    board[i][j] = "O"
                elif board[i][j] == "O":
                    board[i][j] = "X"

- 时间复杂度: O(mn)
- 空间复杂夫: O(mn)

## 1254. 统计封闭岛屿的数目 

感觉也不用并查集

先遍历一遍矩阵把和边相连的陆地区域都变成水，这样就只剩下岛屿，再循环dfs或bfs将联通的岛屿再变成水，直到矩阵中没有陆地，那么循环的次数就是岛屿的数目。

In [79]:
class Solution:
    def closedIsland(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 0:
                        grid[nx][ny] = 1
                        dfs(nx, ny)
        
        m, n = len(grid), len(grid[0])
        for i in range(m):
            if grid[i][0] == 0: 
                grid[i][0] = 1
                dfs(i, 0)
            if grid[i][n - 1] == 0: 
                grid[i][n - 1] = 1
                dfs(i, n - 1)
        for i in range(n):
            if grid[0][i] == 0: 
                grid[0][i] = 1
                dfs(0, i)
            if grid[m - 1][i] == 0: 
                grid[m - 1][i] = 1
                dfs(m - 1, i)
        
        ans = 0
        for i in range(m):
            for j in range(n):
                if grid[i][j] == 0:
                    ans += 1
                    dfs(i, j)
        return ans

In [84]:
grid = [[1,1,1,1,1,1,1],[1,0,0,0,0,0,1],[1,0,1,1,1,0,1],[1,0,1,0,1,0,1],[1,0,1,1,1,0,1],[1,0,0,0,0,0,1],[1,1,1,1,1,1,1]]

In [85]:
s = Solution()
s.closedIsland(grid)

2

- 时间复杂度：O(mn)
- 空间复杂度：O(mn)

## 695. 岛屿的最大面积

In [123]:
class Solution:
    def maxAreaOfIsland(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            cnt = 1  # 设为 1 比设为 0 要好
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 1:
                        grid[nx][ny] = 0
                        cnt += dfs(nx, ny)
            return cnt
        
        m, n = len(grid), len(grid[0])
        ans = 0
        for i in range(m):
            for j in  range(n):
                if grid[i][j] == 1:
                    grid[i][j] = 0
                    ans = max(ans, dfs(i, j) + 1)
        return ans

In [121]:
grid = [[0,0,0,0,0,0,0,0]]

In [122]:
s = Solution()
s.maxAreaOfIsland(grid)

0

## 1905. 统计子岛屿

In [155]:
class Solution:
    def countSubIslands(self, grid1: List[List[int]], grid2: List[List[int]]) -> int:
        def dfs(x, y):
            ans = 1
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid2[nx][ny] == 1:
                        grid2[nx][ny] = 0
                        ans &= (dfs(nx, ny) and grid1[nx][ny])  # 注意这里的短路，要先执行dfs
            # res = ans & grid1[x][y] == 1
            # if res: print(x, y)
            
            return ans & grid1[x][y] == 1
        
        m, n = len(grid2), len(grid2[0])
        num = 0
        for i in range(m):
            for j in range(n):
                if grid2[i][j] == 1:
                    grid2[i][j] = 0
                    num += dfs(i, j)
        
        return num

In [156]:
grid1 = [[1,1,1,0,0],[0,1,1,1,1],[0,0,0,0,0],[1,0,0,0,0],[1,1,0,1,1]]
grid2 = [[1,1,1,0,0],[0,0,1,1,1],[0,1,0,0,0],[1,0,1,1,0],[0,1,0,1,0]]

In [157]:
s = Solution()
s.countSubIslands(grid1, grid2)

3

## 417. 太平洋大西洋水流问题

In [34]:
class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        m, n = len(heights), len(heights[0])

        def search(starts: List[Tuple[int, int]]) -> Set[Tuple[int, int]]:
            visited = set()
            def dfs(x: int, y: int):
                if (x, y) in visited:
                    return
                visited.add((x, y))
                for nx, ny in ((x, y + 1), (x, y - 1), (x - 1, y), (x + 1, y)):
                    if 0 <= nx < m and 0 <= ny < n and heights[nx][ny] >= heights[x][y]:
                        dfs(nx, ny)
            for x, y in starts:
                dfs(x, y)
            return visited

        pacific = [(0, i) for i in range(n)] + [(i, 0) for i in range(1, m)]
        atlantic = [(m - 1, i) for i in range(n)] + [(i, n - 1) for i in range(m - 1)]
        return list(map(list, search(pacific) & search(atlantic)))

### set 的交和并: & |

In [19]:
{1, 2} & {1}

{1}

In [20]:
{1, 2} | {1}

{1, 2}

反向的思路就是：分别从岛的大西洋和太平洋的边界开始进行遍历，分别获得能够流向大西洋以及太平洋的格子的合集，再取交集即为结果。  
而正向做比较困难。

In [40]:
class Solution:
    def pacificAtlantic(self, heights: List[List[int]]) -> List[List[int]]:
        def search(starts):
            visit = set()
            def dfs(x, y):
                visit.add((x, y))
                for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < m and 0 <= ny < n and (nx, ny) not in visit and heights[x][y] <= heights[nx][ny]:
                        dfs(nx, ny)
            for x, y in starts: dfs(x, y)
            return visit
        
        m, n = len(heights), len(heights[0])
        pacific = [(0, i) for i in range(n)] + [(i, 0) for i in range(m)]
        atlantic = [(m - 1, i) for i in range(n)] + [(i, n - 1) for i in range(m - 1)]
        return list(map(list, search(pacific) & search(atlantic)))

In [41]:
heights = [[2,1],[1,2]]

In [42]:
s = Solution()
s.pacificAtlantic(heights)

[[0, 1], [1, 0], [0, 0], [1, 1]]

- 时间复杂度: O(mn)
- 空间复杂度: O(mn)

## 733. 图像渲染

In [45]:
class Solution:
    def floodFill(self, image: List[List[int]], sr: int, sc: int, color: int) -> List[List[int]]:
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if image[nx][ny] == old and image[nx][ny] != color:
                        image[nx][ny] = color
                        dfs(nx, ny)
        m, n, old = len(image), len(image[0]), image[sr][sc]
        image[sr][sc] = color
        dfs(sr, sc)
        return image

## 1020. 飞地的数量

In [2]:
from typing import List
class Solution:
    def numEnclaves(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            grid[x][y] = 0
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if grid[nx][ny] == 1:
                        dfs(nx, ny)
                        
        m, n = len(grid), len(grid[0])
        for i in range(m):
            if grid[i][0] == 1:
                dfs(i, 0)
            if grid[i][n - 1] == 1:
                dfs(i, n - 1)
        for i in range(n):
            if grid[0][i] == 1:
                dfs(0, i)
            if grid[m - 1][i] == 1:
                dfs(m - 1, i)
        
        return sum(r.count(0) for r in grid)

## 684. 冗余连接

检测图中有没有环

In [13]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
    
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return 0
        if self.rank[fx] > self.rank[fy]:
            fx, fy = fy, fx
        self.fa[fx] = self.fa[fy]
        if self.rank[fx] == self.rank[fy]:
            self.rank[fy] += 1
        return 1

class Solution:
    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        uf = UF(len(edges))
        for x, y in edges:
            ans = uf.union(x - 1, y - 1)
            if ans == 0: return [x, y]        

In [14]:
edges = [[1,2], [2,3], [3,4], [1,4], [1,5]]

In [15]:
s = Solution()
s.findRedundantConnection(edges)

[1, 4]

## 685. 冗余连接Ⅱ

### 步骤：
1. 计算入度
2. 根据入度的不同分两种情况：
    - 存在入度为 2 的点： 若删去某条边之后该图变成了一棵树，那么这条边为要返回的结果。
    - 不存在入度为 2 的点： 必然存在有向环， 找到有向环的边就是要返回的答案。

In [30]:
from typing import List

class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
    
    def find(self, x):
        return x if self.fa[x] == x else self.find(self.fa[x])
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return 0
        self.fa[y] = x
        return 1

    
class Solution:
    def findRedundantDirectedConnection(self, edges: List[List[int]]) -> List[int]:
        def toTreeCutEdge(edge):
            uf = UF(n + 1)
            for a, b in edges:
                if [a, b] == edge: continue
                # 去掉这条边以后依然不是树，说明答案不是这条边
                if uf.union(a, b) == 0: return False
            return True
    
        n = len(edges)
        indegree = [0] * (n + 1)
        for _, e in edges: indegree[e] += 1
        
        if 2 in indegree:
            target = indegree.index(2)
            for i in range(n - 1, -1, -1):
                # 入度存在 2， 则要删去入度为 2 的点连接的一条边
                if edges[i][1] == target and toTreeCutEdge(edges[i]):
                    return edges[i]
            
        
        # 入度不存在 2 ，图中存在有向环，找到最后出现的有向环的边即可
        uf = UF(n + 1)
        # for a, b in edges: uf.union(a, b)  # 这里注意不能一开始就全合并，不然很多边的祖先都是同一个
        for a, b in edges:
            if uf.find(a) == uf.find(b):  # 先判断，再合并
                return [a, b]
            uf.union(a, b)

In [31]:
edges = [[1,2],[2,3],[3,4],[4,1],[1,5]]

In [32]:
s = Solution()
s.findRedundantDirectedConnection(edges)

[4, 1]

## 1162. 地图分析

曼哈顿距离不就是遍历的步数吗

In [28]:
from collections import defaultdict
from typing import List
class Solution:
    def maxDistance(self, grid: List[List[int]]) -> int:
        def dfs(x, y, step):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        dist[(nx, ny)] = step if (nx, ny)  not in dist else min(dist[(nx, ny)], step)
                        dfs(nx, ny, step + 1)
                        visit[nx][ny] = 1
        
        n = len(grid)
        dist = dict()
        visit = [[1] * n for _ in range(n)]
        
        for i in range(n):
            for j in range(n):
                if grid[i][j] == 1:
                    dfs(i, j, 1)
        ans = 0
        for key in dist: ans = max(ans, dist[key])
        
        return ans

In [37]:
grid = [[1,0,1],[0,0,0],[1,0,1]]

In [38]:
s = Solution()
s.maxDistance(grid)

2

In [35]:
sum(map(sum, grid))

4

这样做复杂度太高，得多源bfs

### 多源bfs做法

适用于多个等同的起点，实际上就是把所有root节点都先入栈。

In [36]:
from collections import deque
class Solution:
    def maxDistance(self, grid: List[List[int]]) -> int:
        stk = deque()
        n = len(grid)
        for i in range(n):
            for j in range(n):
                if grid[i][j] == 1: stk.append((i, j, 0))
        l = len(stk)
        if l == 0 or l == n * n: return -1
    
        visit = [[1] * n for _ in range(n)]
        while stk:
            cx, cy, step = stk.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = cx + dx, cy + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        stk.append((nx, ny, step + 1))
        return step

### 通俗解释： 陆地逐渐扩张，直到铺满整个区域！

## 42. 接雨水

### 1. dp

一个格子能够接住雨水的最大高度为两边最大高度的最小值。

In [62]:
class Solution:
    def trap(self, height: List[int]) -> int:
        def rightmax(idx):
            if idx in memo: return memo[idx]
            if idx == n - 1: return height[n - 1]
            memo[idx] = max(height[idx], rightmax(idx + 1))
            return memo[idx]
        
        n, ans, memo = len(height), 0, {}
        leftmax = height[0]
        for i in range(n):
            leftmax = max(leftmax, height[i])
            ans += min(leftmax, rightmax(i)) - height[i]
        
        return ans

In [66]:
height = [4,2,0,3,2,5]

In [67]:
s = Solution()
s.trap(height)

9

In [65]:
class Solution:
    def trap(self, height: List[int]) -> int:
        n = len(height)
        rightmax = [0] * n
        cur = 0
        for i in range(n - 1, -1, -1):
            cur = max(cur, height[i])
            rightmax[i] = cur
        leftmax = 0
        ans = 0
        for i in range(n):
            leftmax = max(leftmax, height[i])
            ans += min(leftmax, rightmax[i]) - height[i]
        return ans

### 2. 单调栈

In [68]:
class Solution:
    def trap(self, height: List[int]) -> int:
        ans = 0
        stack = list()
        n = len(height)
        
        for i, h in enumerate(height):
            while stack and h > height[stack[-1]]:
                top = stack.pop()
                if not stack:
                    break
                left = stack[-1]
                currWidth = i - left - 1
                currHeight = min(height[left], height[i]) - height[top]
                ans += currWidth * currHeight
            stack.append(i)
        
        return ans

## 542. 01矩阵

和1162类似, 多源BFS

In [10]:
from collections import deque
class Solution:
    def updateMatrix(self, mat: List[List[int]]) -> List[List[int]]:
        m, n = len(mat), len(mat[0])
        stk = deque()
        # dist = [[float("Inf")] * n for _ in range(m)]
        for i in range(m):
            for j in range(n):
                if mat[i][j] == 0: 
                    # dist[i][j] = 0
                    stk.append((i, j, 0))
        
        visit = [[1] * n for _ in range(m)]
        while stk:
            x, y, step = stk.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if mat[nx][ny] == 1 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        mat[nx][ny] = step + 1
                        stk.append((nx, ny, step + 1))
                        
        return mat

In [11]:
mat = [[0,0,0],[0,0,0],[0,0,0]]

In [12]:
s = Solution()
s.updateMatrix(mat)

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

## 1091. 二进制矩阵中的最短路径

dp吧

In [69]:
class Solution:
    def shortestPathBinaryMatrix(self, grid: List[List[int]]) -> int:
        if grid[0][0] != 0 or grid[-1][-1] != 0: return -1
        n, stk = len(grid), deque([(0, 0, 1)])
        visit = [[1] * n for _ in range(n)]
        while stk:
            x, y, step = stk.popleft()
            # print(x, y, type(x)
            if x == y == n - 1: return step
            d = ((-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1))
            for dx, dy in d:
                # print(dx, dy, type(dx))
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 0 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        stk.append((nx, ny, step + 1))
        return -1

In [70]:
grid = [[0,0,0],[1,1,0],[1,1,0]]

In [71]:
s = Solution()
s.shortestPathBinaryMatrix(grid)

4

### 在二维矩阵中搜索，什么时候用BFS，什么时候用DFS？

1. 如果只是要找到某一个结果是否存在，那么DFS会更高效。因为DFS会首先把一种可能的情况尝试到底，才会回溯去尝试下一种情况，只要找到一种情况，就可以返回了。但是BFS必须所有可能的情况同时尝试，在找到一种满足条件的结果的同时，也尝试了很多不必要的路径；
2. 如果是要找所有可能结果中最短的，那么BFS会更高效。因为DFS是一种一种地尝试，在把所有可能的情况试完之前，无法确定哪个是最短。所以DFS必须把所有的情况都找一遍，才能确定最终答案（DFS的优化就是剪枝，不剪枝很容易超时）。而BFS从一开始就是尝试所有情况，所以只要找到第一个达到的那个点，那就是最短的路径，可以直接返回，其他的情况都可以省略了，所以这种情况下，BFS更高效。

### BFS解法中的visited为什么可以全局使用？

BFS是在尝试所有可能的路径，哪个最快到达终点，哪个就是最短。那么每一条路径走过的路不同，visited也应该不同，为什么visited可以全局使用呢？  
因为我们要找的是最短路径，那么如果在此之前某个点已经在visited中，也就是说有其他路径在小于或等于当前步数的情况下，到达过这个点，证明到达这个点的最短路径已经被找到。那么显然这个点没必要再尝试了，因为即便去尝试了，最终的结果也不会是最短路径了，所以直接放弃这个点即可。

## 934. 最短的桥

先遍历完一座岛，再对这个岛上的每个点使用BFS

In [26]:
from collections import deque
from typing import List

class Solution:
    def shortestBridge(self, grid: List[List[int]]) -> int:
        def dfs(x, y):
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if grid[nx][ny] == 1 and visit[nx][ny]:
                        visit[nx][ny] = 0
                        island.append((nx, ny, 0))
                        dfs(nx, ny)
        
        n = len(grid)
        visit = [[1] * n for _ in range(n)]
        island = deque()
        for i in range(n):
            break1 = 0
            for j in range(n):
                if grid[i][j] == 1:
                    island.append((i, j, 0))
                    visit[i][j] = 0
                    dfs(i, j)
                    break1 = 1
                    break
            if break1: break
        
        while island:
            # print(island)
            x, y, step = island.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < n and 0 <= ny < n:
                    if visit[nx][ny]:
                        visit[nx][ny] = 0
                        if grid[nx][ny] == 1: return step
                        grid[nx][ny] = 1
                        island.append((nx, ny, step + 1))
        
        return -1

In [27]:
grid = [[1,1,1,1,1],[1,0,0,0,1],[1,0,1,0,1],[1,0,0,0,1],[1,1,1,1,1]]

In [28]:
s = Solution()
s.shortestBridge(grid)

1

## 1926. 迷宫中离入口最近的出口

关键词：最近，因此还是使用BFS

In [36]:
from collections import deque
class Solution:
    def nearestExit(self, maze: List[List[str]], entrance: List[int]) -> int:
        m, n = len(maze), len(maze[0])
        sx, sy = entrance[0], entrance[1]
        stk = deque([(sx, sy, 0)])
        visit = [[1] * n for _ in range(m)]
        visit[sx][sy] = 0
        
        while stk:
            x, y, step = stk.popleft()
            for dx, dy in ((-1, 0), (1, 0), (0, -1), (0, 1)):
                nx, ny = x + dx, y + dy
                if 0 <= nx < m and 0 <= ny < n:
                    if maze[nx][ny] == "." and visit[nx][ny]:
                        if nx == 0 or ny == 0 or nx == m - 1 or ny == n - 1:
                            return step + 1
                        visit[nx][ny] = 0
                        stk.append((nx, ny, step + 1))
        
        return -1

In [39]:
maze = [["+",".","+","+","+","+","+"],["+",".","+",".",".",".","+"],["+",".","+",".","+",".","+"],["+",".",".",".","+",".","+"],["+","+","+","+","+",".","+"]]
entrance = [0,1]

In [40]:
s = Solution()
s.nearestExit(maze, entrance)

12

## 841. 钥匙和房间

此为判断解是否存在的类型，应用DFS较方便

In [20]:
class Solution:
    def canVisitAllRooms(self, rooms: List[List[int]]) -> bool:
        def dfs(num: int, cnt: int) -> int:
            print(num, cnt)
            if cnt == n: return True
            ans = False
            for nxt in rooms[num]:
                if visit[nxt]:
                    visit[nxt] = 0
                    ans |= dfs(nxt, cnt + 1)
                    visit[nxt] = 1
            return ans
        
        n = len(rooms)
        visit = [1] * n
        visit[0] = 0
        return dfs(0, 1)

In [40]:
rooms = [[1],[2],[3],[]]

In [41]:
s = Solution()
s.canVisitAllRooms(rooms)

True

理解错题意了，上述是判断是否有一条路可以遍历完所有房间。  
此题还是考察是否能从根节点遍历到所有节点，用BFS思想更好

In [39]:
from collections import deque
class Solution:
    def canVisitAllRooms(self, rooms: List[List[int]]) -> bool:
        n, stk = len(rooms), deque([0])
        visit = [1] * n
        while stk:
            # print(visit)
            # 说明遍历完了
            cur = stk.popleft()
            if visit[cur]:
                visit[cur] = 0
                stk.extend(rooms[cur])
            if 1 not in visit: return True
        
        return False  

## 547. 省份数量

回顾一下并查集

In [54]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
    
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            self.fa[fx] = fy
            if self.rank[fx] == self.rank[fy]:
                self.rank[fy] += 1
    

class Solution:
    def findCircleNum(self, isConnected: List[List[int]]) -> int:
        n = len(isConnected)
        uf = UF(n)
        for i in range(n - 1):
            for j in range(i + 1, n):
                if isConnected[i][j]:
                    uf.union(i, j)
        # return len(Counter(uf.fa))
        # return uf.fa
        return sum(uf.fa[i] == i for i in range(n))

In [55]:
isConnected = [[1,0,0,1],[0,1,1,0],[0,1,1,1],[1,0,1,1]]

In [56]:
s = Solution()
s.findCircleNum(isConnected)

1

## 1319. 连通网络的操作次数

In [73]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.rank = [1] * n
        self.extra = 0  # 记录多余的边的数量
        
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx != fy:
            if self.rank[fx] > self.rank[fy]:
                fx, fy = fy, fx
            self.fa[fx] = fy
            if self.rank[fx] == self.rank[fy]:
                self.rank[fy] += 1
        else: self.extra += 1  # 若两个元素的父节点相同则代表他们构成的图已经连通，此条边即为多余的边

class Solution:
    def makeConnected(self, n: int, connections: List[List[int]]) -> int:
        uf = UF(n)
        for x, y in connections:
            uf.union(x, y)
        num = sum(uf.fa[i] == i for i in range(n)) - 1  # 统计连接起图中的所有集合需要多少条边
        return max(num, 0) if uf.extra >= num else -1  # 这里单独处理了一下所有节点全部连通的情况，即图中只有一个集合，则返回 0

In [76]:
n = 4
connections = [[0,1],[0,2],[1,2]]

In [77]:
s = Solution()
s.makeConnected(n, connections)

1

## 1376. 通知所有员工所需的时间

In [112]:
class Solution:
    def numOfMinutes(self, n: int, headID: int, manager: List[int], informTime: List[int]) -> int:
        g = [[] for _ in range(n)]
        for i, num in enumerate(manager):
            # print(i, num)
            if num == -1: continue
            g[i].append(num)
            g[num].append(i)
        
        # print(g)
        
        stk = deque([(headID, 0)])
        ans = 0
        vis = [1] * n
        vis[headID] = 0
        while stk:
            cur, curt = stk.popleft()
            curt += informTime[cur]
            ans = max(ans, curt)
            for nxt in g[cur]:
                if vis[nxt]:
                    vis[nxt] = 0
                    stk.append((nxt, curt))
                    
        return ans

In [113]:
n = 6
headID = 2
manager = [2,2,-1,2,2,2]
informTime = [0,0,1,0,0,0]

In [114]:
s = Solution()
s.numOfMinutes(n, headID, manager, informTime)

1

## 802. 找到最终的安全状态

dp + dfs + 回溯 ？

In [138]:
class Solution:
    def eventualSafeNodes(self, graph: List[List[int]]) -> List[int]:
        n = len(graph)
        vis = [1] * n
        memo = {}
        
        def dp(num):
            if num in memo: return memo[num]
            if not graph[num]: 
                memo[num] = 1
                return 1
            ans = 1
            for nxt in graph[num]:
                if not vis[nxt]: 
                    memo[num] = 0
                    return memo[num]
                vis[nxt] = 0
                ans = ans and dp(nxt)
                vis[nxt] = 1
            
            memo[num] = ans
            return memo[num]
            
        res = []
        for i in range(n):
            if dp(i): res.append(i)
            
        return res
        
        

In [139]:
graph = [[1,2],[2,3],[5],[0],[5],[],[]]

In [140]:
s = Solution()
s.eventualSafeNodes(graph)

[2, 4, 5, 6]

### 拓扑排序 （再议）

## 1129. 颜色交替的最短路径

In [2]:
from typing import List
class Solution:
    def shortestAlternatingPaths(self, n: int, redEdges: List[List[int]], blueEdges: List[List[int]]) -> List[int]:
        ans = [-1] * n
        red_neighbors = defaultdict(set)
        blue_neighbors = defaultdict(set)

        def generate_neighbors(neighbors, edges):
            for i, j in edges:
                neighbors[i].add(j)

        generate_neighbors(red_neighbors, redEdges)
        generate_neighbors(blue_neighbors, blueEdges)

        seen = set()
        start_0 = (0, 0) # pos, color 0:blue 1:red
        start_1 = (0, 1)
        seen.add(start_0)
        seen.add(start_1)
        q = deque([start_0, start_1])
        step = 0
        while q:
            size = len(q)
            for _ in range(size):
                node, pre_color = q.popleft()
                if ans[node] == -1:
                    ans[node] = step
                else:
                    ans[node] = min(ans[node], step)
                if pre_color == 0:  # blue, search red_edges
                    for next_node in red_neighbors[node]:
                        if (next_node, 1) not in seen:
                            seen.add((next_node, 1))
                            q.append((next_node, 1))
                elif pre_color == 1:
                    for next_node in blue_neighbors[node]:
                        if (next_node, 0) not in seen:
                            seen.add((next_node, 0))
                            q.append((next_node, 0))
            step += 1
        return ans

## 1466. 重新规划路线

In [9]:
from collections import deque
from typing import List
class Solution:
    def minReorder(self, n: int, connections: List[List[int]]) -> int:
        g = [[] for _ in range(n)]
        for x, y in connections:
            g[x].append((y, 0))  # 图中已有的边，cost 0
            g[y].append((x, 1))  # 反向边， cost 1
        
        visit = [1] * n
        visit[0] = 0
        stk = deque([nxt for nxt in g[0]])
        cost = 0  # 总体的cost
        while stk:
            cur, c = stk.popleft()
            if visit[cur]:
                cost += c
                visit[cur] = 0
                for nxt in g[cur]:
                    stk.append(nxt)
        
        return n - 1 - cost  # 边数减去cost即为答案

In [14]:
n = 3
connections = [[1,0],[2,0]]

In [15]:
s = Solution()
s.minReorder(n, connections)

0

In [28]:
class Solution:
    def canReach(self, arr: List[int], start: int) -> bool:
        def dfs(idx):
            if idx in memo: return memo[idx]
            if arr[idx] == 0: 
                memo[idx] = True
                return memo[idx]
            ans = False
            
            for i in (-arr[idx], arr[idx]):
                nidx = idx + i
                if 0 <= nidx < n and visit[nidx]:
                    visit[nidx] = 0
                    ans = ans or dfs(nidx)
                    visit[nidx] = 1
            
            memo[idx] = ans
            return memo[idx]
        
        n = len(arr)
        visit = [1] * n
        memo = {}
        dfs(start)
        return memo[start]
        # return memo

In [40]:
arr = [3,0,2,1,2]
start = 2

In [41]:
s = Solution()
s.canReach(arr, start)

False

In [39]:
class Solution:
    def canReach(self, arr: List[int], start: int) -> bool:
        def dfs(idx):
            if arr[idx] == 0: return True
            ans = False
            for i in (-arr[idx], arr[idx]):
                nidx = idx + i
                if 0 <= nidx < n and nidx not in visit:
                    visit.add(nidx)
                    ans = ans or dfs(nidx)
            return ans
        
        n = len(arr)
        visit = {start}
        return dfs(start)

In [42]:
class Solution:
    def canReach(self, arr: List[int], start: int) -> bool:
        if arr[start] == 0:
            return True

        n = len(arr)
        used = {start}
        q = collections.deque([start])

        while len(q) > 0:
            u = q.popleft()
            for v in [u + arr[u], u - arr[u]]:
                if 0 <= v < n and v not in used:
                    if arr[v] == 0:
                        return True
                    q.append(v)
                    used.add(v)
        
        return False

## 1654. 到家的最少跳跃次数

In [74]:
from collections import deque
class Solution:
    def minimumJumps(self, forbidden: List[int], a: int, b: int, x: int) -> int:
        forbidden = set(forbidden)
        vis = {(0, 0)}
        stk = deque([(0, 0, 0)])
        while stk:
            loc, step, back = stk.popleft()
            if loc == x: return step
            f = loc + a
            if f not in forbidden:
                if (loc, f) in vis: return -1
                vis.add((loc, f))
                stk.append((f, step + 1, 0))
            
            if back == 0:
                ba = loc - b
                if ba not in forbidden and ba >= 0:
                    # if (loc, ba) in vis: return -1
                    vis.add((loc, ba))
                    stk.append((ba, step + 1, 1))
        
        return -1

In [75]:
forbidden = [8,3,16,6,12,20]
a = 15
b = 13
x = 11

In [76]:
s = Solution()
s.minimumJumps(forbidden, a, b, x)

-1

关键在于终止条件：最优解不超过最远位置（$ max(max(forbidden) + a, + b, x + b) $）。而不是重复到达某个位置。

## 365. 水壶问题

In [88]:
from collections import deque
class Solution:
    def canMeasureWater(self, jug1Capacity: int, jug2Capacity: int, targetCapacity: int) -> bool:
        jug1, jug2 = jug1Capacity, jug2Capacity
        vis = {(0, 0)}
        stk = deque([(0, 0)])
        while stk:
            a, b = stk.popleft()
            print(a, b)
            sum_ = a + b
            nxt = ((jug1, b), (a, jug2), (0, b), (a, 0), 
                   (min(jug1, sum_), max(sum_ - jug1, 0)), 
                   (min(jug2, sum_, ), max(sum_ - jug2, 0)))
            for x, y in nxt:
                if x + y == targetCapacity: return True
                if (x, y) not in vis:
                    vis.add((x, y))
                    stk.append((x, y))
        
        return False

In [110]:
jug1Capacity = 2
jug2Capacity = 6
targetCapacity = 5

In [111]:
s = Solution()
s.canMeasureWater(jug1Capacity, jug2Capacity, targetCapacity)

2 6
0 0 0
2 0 1
0 6 1
2 6 2
0 2 2
2 4 2
2 2 3
0 4 3


False

In [107]:
from collections import deque
class Solution:
    def canMeasureWater(self, jug1Capacity: int, jug2Capacity: int, targetCapacity: int) -> bool:
        jug1, jug2 = jug1Capacity, jug2Capacity
        print(jug1, jug2)
        vis = {(0, 0)}
        stk = deque([(0, 0, 0)])
        while stk:
            a, b, step = stk.popleft()
            print(a, b, step)
            sum_ = a + b
            nxt = ((jug1, b), (a, jug2), (0, b), (a, 0), 
                   (min(jug1, sum_), max(sum_ - jug1, 0)), 
                   (max(sum_ - jug2, 0), min(jug2, sum_)))
            # print(nxt)
            for x, y in nxt:
                if x + y == targetCapacity: return True
                if (x, y) not in vis:
                    vis.add((x, y))
                    stk.append((x, y, step + 1))
        
        return False

## 752. 打开转盘锁

In [23]:
from collections import deque
from typing import List
class Solution:
    def openLock(self, deadends: List[str], target: str) -> int:
        deadends = set(deadends)
        if "0000" in deadends: return -1
        vis = {"0000"}
        stk = deque([("0000", 0)])
        while stk:
            cur, step = stk.popleft()
            if cur == target: return step
            for i, ch in enumerate(cur):
                f, b = cur[:i], cur[i + 1:]
                for j in (1, 9):
                    c = str((int(ch) + j) % 10)
                    nxt = f + c + b
                    # print(nxt)
                    if nxt not in deadends and nxt not in vis:
                        vis.add(nxt)
                        stk.append((nxt, step + 1))
        return -1

In [24]:
deadends = ["8887","8889","8878","8898","8788","8988","7888","9888"]
target = "8888"

In [25]:
s = Solution()
s.openLock(deadends, target)

-1

## 997. 找到小镇的法官

In [2]:
from typing import List
class Solution:
    def findJudge(self, n: int, trust: List[List[int]]) -> int:
        g = [[] for _ in range(n + 1)]
        cnt = [0] * (n + 1)
        for a, b in trust:
            g[a].append(b)
            cnt[b] += 1
        for i in range(1, n + 1):
            if not g[i] and cnt[i] == n - 1:
                return i
        return -1

## 1557. 可以达到所有点的最少点数目

In [40]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        # self.rank = [1] * n
    
    def find(self, x):
        if x != self.fa[x]:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        if self.fa[x] == x:
            self.fa[x] = y

class Solution:
    def findSmallestSetOfVertices(self, n: int, edges: List[List[int]]) -> List[int]:
        uf = UF(n)
        for a, b in edges:
            uf.union(b, a)
        
        res = []
        for i in range(n):
            if uf.fa[i] == i:
                res.append(i)
        print(uf.fa)
        return res

In [45]:
n = 5
edges = [[0,1],[2,1],[3,1],[1,4],[2,4]]

In [46]:
s = Solution()
s.findSmallestSetOfVertices(n, edges)

[0, 2, 3]

In [44]:
class Solution:
    def findSmallestSetOfVertices(self, n: int, edges: List[List[int]]) -> List[int]:
        fa = [i for i in range(n)]
        for a, b in edges:
            if fa[b] == b:
                fa[b] = a
        
        res = []
        for i, n in enumerate(fa):
            if i == n: res.append(i)
        
        return res

## 1615. 最大网络秩

In [20]:
from typing import List
class Solution:
    def maximalNetworkRank(self, n: int, roads: List[List[int]]) -> int:
        g = [set() for _ in range(n)]
        for a, b in roads:
            g[a].add(b)
            g[b].add(a)
        
        l = [len(g[i]) for i in range(n)]
        
        ans = 0
        for i in range(n):
            for j in range(i + 1, n):
                cur = l[i] + l[j]
                if j in g[i]: cur -= 1
                ans = max(ans, cur)
        
        # print(g)
        
        return ans

In [21]:
n = 8
roads = [[0,1],[1,2],[2,3],[2,4],[5,6],[5,7]]

In [22]:
s = Solution()
s.maximalNetworkRank(n, roads)

5

## 886. 可能的二分法

试一下并查集解: 将一个人不喜欢的人都归类为同一个集合，若发现这个人与不喜欢的人在同一个集合则无合理划分

In [25]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.r = [1] * n
        
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return
        if self.r[fx] > self.r[fy]:
            fx, fy = fy, fx
        self.fa[fx] = fy
        if self.r[fx] == self.r[fy]:
            self.r[fy] += 1
    
    def is_connected(self, x, y):
        return self.fa[x] == self.fa[y]
    
class Solution:
    def possibleBipartition(self, n: int, dislikes: List[List[int]]) -> bool:
        g = [[] for _ in range(n)]
        for a, b in dislikes:
            a, b = a - 1, b - 1
            g[a].append(b)
            g[b].append(a)
        
        uf = UF(n)
        for x, dis in enumerate(g):
            for y in dis:
                uf.union(dis[0], y)
                if uf.is_connected(x, y): return False
        return True

## 785. 判断二分图

### 并查集

In [26]:
class UF:
    def __init__(self, n):
        self.fa = [i for i in range(n)]
        self.r = [1] * n
        
    def find(self, x):
        if self.fa[x] != x:
            self.fa[x] = self.find(self.fa[x])
        return self.fa[x]
    
    def union(self, x, y):
        fx, fy = self.find(x), self.find(y)
        if fx == fy: return
        if self.r[fx] > self.r[fy]:
            fx, fy = fy, fx
        self.fa[fx] = fy
        if self.r[fx] == self.r[fy]:
            self.r[fy] += 1
    
    def is_connected(self, x, y):
        return self.fa[x] == self.fa[y]

class Solution:
    def isBipartite(self, graph: List[List[int]]) -> bool:
        uf = UF(len(graph))
        for x, nxt in enumerate(graph):
            for n in nxt:
                uf.union(nxt[0], n)
                if uf.is_connected(x, n): return False
        return True

### 染色法

In [28]:
class Solution:
    def isBipartite(self, graph: List[List[int]]) -> bool:
        def dfs(i, c):
            color[i] = c
            for n in graph[i]:
                if color[n] == c: return False
                if color[n] == 0 and not dfs(n, -c): return False
            return True
        
        color = [0] * len(graph)
        return all(c or dfs(i, 1) for i, c in enumerate(color))